In [153]:
# Initialisation
import os
import io
import requests
import json
import pandas as pd
import urllib.parse as url
import alpaca_trade_api as tradeapi
#import matplotlib.pyplot as plt
from dotenv import load_dotenv

In [154]:
# Initialised
import panel as pn
from panel.interact import interact
from panel import widgets
pn.extension()

In [155]:
import hvplot.pandas

In [156]:
load_dotenv()
api_key = os.getenv("DOMAIN_API_KEY")
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
alpaca_api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [157]:
#%matplotlib inline

In [158]:
title = "All In One Portfolio Summary and Analysis"

In [159]:
# REQUEST CALL DOMAIN.COM.AU: This api call will, given a string, finds the property address that closest matches the string
# and returns the address in full as a string, each address components in its own string, and a unique ID for the given address.
# We will return all this data in a df
def address_suggest(address):
    if not address:
        return
    address_url = url.quote(address)
    request_url = f"https://api.domain.com.au/v1/properties/_suggest?terms={address_url}&pageSize=1&api_key={api_key}"
    get_request = requests.get(request_url)

    # Data formatting
    data = get_request.json()
    # Extract data
    full_address = data[0]["address"]
    address_data = data[0]["addressComponents"]
    location_id = data[0]["id"]
    
    
    # Address_data is already a dict so we need to combine everything with update
    all_data = {
        "full_address": full_address,
        "id": location_id
    }
    all_data.update(address_data)
    # Put data into df
    # Df will have a single row. All data in their own columns with their own names
    address_df = pd.DataFrame([all_data])
    return address_df

In [160]:
# Address locators: this request will return a unique identifier for the following address levels:
# Address, street, suburb, postcode

def address_locators(address_df, level, building_type):
    if building_type == "unit":
        unit = address_df.iloc[0]["unitNumber"]
    else:
        unit = None
    street_num = address_df.iloc[0]["streetNumber"]
    street_name = address_df.iloc[0]["streetName"]
    street_type = address_df.iloc[0]["streetType"]
    suburb = address_df.iloc[0]["suburb"]
    post_code = address_df.iloc[0]["postCode"]
    state = address_df.iloc[0]["state"]
    
    # Check if building is actually a unit
    if building_type == "unit":
        request_url = f"https://api.domain.com.au/v1/addressLocators?searchLevel={level}&unitNumber={unit}&streetNumber={street_num}&streetName={street_name}&streetType={street_type}&suburb={suburb}&state={state}&postcode={post_code}&api_key={api_key}"

    else:
        request_url = f"https://api.domain.com.au/v1/addressLocators?searchLevel={level}&streetNumber={street_num}&streetName={street_name}&streetType={street_type}&suburb={suburb}&state={state}&postcode={post_code}&api_key={api_key}"


    get_request = requests.get(request_url)

    # Data formatting
    data = get_request.json()
    combined_id_data = {}
    for loc_id in data[0]["ids"]:
        combined_id_data.update({loc_id["level"]+" id": loc_id["id"]})
    
    id_df = pd.DataFrame([combined_id_data])
    
    return id_df

In [161]:
# Properties: Using a given property id provided by _suggest, this request gives an info dump on the characteristics of the property
# For now just returning all data since I do not know exactly what data is required
def properties(prop_id):
    request_url = f"https://api.domain.com.au/v1/properties/{prop_id}?api_key={api_key}"
    get_request = requests.get(request_url)
    data = get_request.json()
    return data

In [162]:
# Location profiles: this request gives an info dump of a given location based on the suburb id given. 
# ID must come from Address Locator request
def location_profiles(suburb_id):
    request_url = f"https://api.domain.com.au/v1/locations/profiles/{suburb_id}?api_key={api_key}"
    get_request = requests.get(request_url)
    data = get_request.json()
    return data

In [163]:
"""# Line chart code from previous homework
def create_line_chart(data, title, xlabel, ylabel):
    return data.hvplot.line(
        title=title, 
        xlabel=xlabel, 
        ylabel=ylabel
    ).opts(
        # Format Y to not use scientific notation and increase width so the chart doesn't look squished 
        yformatter="%.0f",
        frame_width = 500
    )"""

'# Line chart code from previous homework\ndef create_line_chart(data, title, xlabel, ylabel):\n    return data.hvplot.line(\n        title=title, \n        xlabel=xlabel, \n        ylabel=ylabel\n    ).opts(\n        # Format Y to not use scientific notation and increase width so the chart doesn\'t look squished \n        yformatter="%.0f",\n        frame_width = 500\n    )'

In [164]:
def suburb_price_chart(growth_df):
    return growth_df.hvplot.line(
        title = "Suburb's growth yearly",
        x = "Years",
        y = "Median Sold Price",
    ).opts(
        yformatter="%.0f"
    )

In [58]:
def suburb_sale_chart(growth_df):
    return growth_df.hvplot.line(
        title = "Suburb's growth yearly",
        x = "Years",
        y = "Number of Sales",
    ).opts(
        yformatter="%.0f"
    )

In [74]:
def purchase_vs_median(growth_df):
    print(growth_df)
    return growth_df.hvplot.bar(
        title = "Purchase Price vs Median Sold Price",
        x = ["Purchase Price"]
    ).opts(
        yformatter="%.0f"
    )

In [72]:
# Top level of property section will do api calls on _suggest and address-locators, determine if property is a unit
# and will produce an info dump on the selected property

def property_top_level(address, valuation):
    if not address or not valuation:
        return
    
    if not valuation.isdigit():
        return pn.Pane("Valuation is not a number")
    
    valuation = int(valuation)
    
    address_df = address_suggest(address)
    full_address = address_df.iloc[0]["full_address"]
    address_string = f"Showing data for the following address:  <br /> {full_address}"
    
    # Create a flag here on top level to indicate if property is a unit
    # unit number wont be a column in address_df if the property isn't a unit to begin with
    if address_df.iloc[0]["unitNumber"]:
        building_type = "unit"
    else:
        building_type = "house"
       
    # Get ids of address componenets
    id_df = address_locators(address_df, "address", building_type)
    
    # Get information on property. Need to know how many bedrooms property has
    property_data = properties(address_df.iloc[0]["id"])
    bedrooms = property_data["bedrooms"]
    info_string = f"This property is a {building_type}, with {bedrooms} bedrooms"
    
    # Get suburbs location profile
    location_data = location_profiles(id_df.iloc[0]["Suburb id"])
    categorised_location_data = location_data["data"]["propertyCategories"]
    
    # Single out the property type by building type and bedroom count
    for property_type in categorised_location_data:
        if property_type["bedrooms"] == bedrooms and property_type["propertyCategory"].lower() == building_type:
            specified_location_data = property_type
    
    # Code here uses specified location data for analysis
    growth_years = []
    growth_price = []
    growth_sales = []
    purchase_price = []
    for entry in specified_location_data["salesGrowthList"]:
        growth_years.append(entry["year"])
        growth_price.append(entry["medianSoldPrice"])
        growth_sales.append(entry["numberSold"])
        purchase_price.append(valuation)
    
    print(purchase_price)
    growth_data = {"Years": growth_years,
                  "Median Sold Price": growth_price,
                  "Number of Sales": growth_sales,
                  "Purchase Price" : purchase_price
                  }
    
    growth_df = pd.DataFrame(growth_data)
    #growth_df.set_index("Years", inplace=True)
    
    
    suburb_data_row = pn.Row(suburb_price_chart(growth_df), suburb_sale_chart(growth_df), purchase_vs_median(growth_df))
    
    test_column = pn.Column(info_string, suburb_data_row)
    
    
    return pn.Column(address_string, test_column, width = 1000)

In [168]:
@interact(x=[2,3,4], y=['a','b','c'])
def func(x,y):
    #print(x,y)
    return pn.Row(x,y)

In [169]:
@interact(x=widgets.TextInput(name="Your Address", placeholder="Please enter your address here"))#, y=widgets.TextInput(name="Your Address", placeholder="Please enter your address here"))
def myfunc(x):
    return x
myfunc

Column
    [0] Column
        [0] TextInput(name='Your Address', placeholder='Please enter y...)
    [1] Row
        [0] Markdown(str, name='interactive05568')

In [170]:
count = 0
count

0

In [171]:
def stock_top_level(stocks):
    if stocks is None:
        return
    stock_df = pd.read_csv(io.BytesIO(stocks), index_col="symbol", infer_datetime_format=True)
    stock_df["date"] = pd.to_datetime(stock_df["date"], infer_datetime_format=True)
    user_symbols = stock_df.index.tolist()
    start_date = stock_df["date"].tolist()
    start_date.sort()
    start_date = start_date[0]
    
    market_data_df = alpaca_api.get_barset(
        user_symbols,
        "1D",
        end = start_date,
        limit = 1000
    ).df
        
    return pn.Pane(market_data_df)

In [35]:
# Put everything together into this dashboard and serve it
property_opening_message = """Please enter your Full Address and Purchase Price  <br />
This program will have you understand how is your property tracking in the current market"""

property_price_message = """Please enter your purchase price <br />
This program will give the purchase price"""

#property_opening_column = pn.Column(property_opening_message,
#                                   interact(property_top_level, 
#                                            address=widgets.TextInput(name="Your Address", placeholder="Please enter your address here")),
#                                   width=600)
#@interact(address=widgets.TextInput(name="Your Address")
property_opening_column = pn.Column(property_opening_message,
                                  interact(property_top_level, 
                                        address=widgets.TextInput(name="Your Address", placeholder="Please enter your address here"),
                                        valuation=widgets.TextInput(name="Your Address", placeholder="Please enter your address here")),
                                   width=600)

stock_opening_message = """Please upload a .csv file with your stock symbol, purchase date, and purchase amount  <br />
Please use the following column names in your csv: symbol, date, amount"""

stock_analysis_opening_column = pn.Column(stock_opening_message, 
                                          interact(stock_top_level, stocks=widgets.FileInput(accept=".csv")),
                                         width = 600)

tabs = pn.Tabs(
    ("Property Analysis", 
     property_opening_column),
    ("Stock Portfolio Analysis",
    stock_analysis_opening_column)
)   
dashboard = pn.Column(title, tabs)
dashboard.servable()

Traceback (most recent call last):
 File "C:\Users\Kevin\anaconda3\envs\pyvizenv\lib\site-packages\pyviz_comms\__init__.py", line 325, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\Kevin\anaconda3\envs\pyvizenv\lib\site-packages\panel\viewable.py", line 273, in _on_msg
 doc.unhold()
 File "C:\Users\Kevin\anaconda3\envs\pyvizenv\lib\site-packages\bokeh\document\document.py", line 799, in unhold
 self.callbacks.unhold()
 File "C:\Users\Kevin\anaconda3\envs\pyvizenv\lib\site-packages\bokeh\document\callbacks.py", line 396, in unhold
 self.trigger_on_change(event)
 File "C:\Users\Kevin\anaconda3\envs\pyvizenv\lib\site-packages\bokeh\document\callbacks.py", line 373, in trigger_on_change
 invoke_with_curdoc(doc, event.callback_invoker)
 File "C:\Users\Kevin\anaconda3\envs\pyvizenv\lib\site-packages\bokeh\document\callbacks.py", line 408, in invoke_with_curdoc
 return f()
 File "C:\Users\Kevin\anaconda3\envs\pyvizenv\lib\site-packages\bokeh\util\callback_manager.py", line 191, in invoke
 callback(attr, old, new)
 File "C:\Users\Kevin\anaconda3\envs\pyvizenv\lib\site-packages\panel\reactive.py", line 302, in _comm_change
 self._process_events({attr: new})
 File "C:\Users\Kevin\anaconda3\envs\pyvizenv\lib\site-packages\panel\reactive.py", line 263, in _process_events
 self.param.set_param(**self_events)
 File "C:\Users\Kevin\anaconda3\envs\pyvizenv\lib\site-packages\param\parameterized.py", line 1904, in set_param
 return self_.update(kwargs)
 File "C:\Users\Kevin\anaconda3\envs\pyvizenv\lib\site-packages\param\parameterized.py", line 1877, in update
 self_._batch_call_watchers()
 File "C:\Users\Kevin\anaconda3\envs\pyvizenv\lib\site-packages\param\parameterized.py", line 2038, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "C:\Users\Kevin\anaconda3\envs\pyvizenv\lib\site-packages\param\parameterized.py", line 2000, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "C:\Users\Kevin\anaconda3\envs\pyvizenv\lib\site-packages\panel\interact.py", line 190, in update_pane
 new_object = self.object(**self.kwargs)
 File "C:\Users\Kevin\AppData\Local\Temp/ipykernel_10428/2732908493.py", line 59, in property_top_level
 purchase_vs_median(growth_df)
 File "C:\Users\Kevin\AppData\Local\Temp/ipykernel_10428/1401233683.py", line 5, in purchase_vs_median
 y = "Median Sold Price",
 File "C:\Users\Kevin\anaconda3\envs\pyvizenv\lib\site-packages\hvplot\plotting\core.py", line 353, in bar
 return self(x, y, kind='bar', **kwds)
 File "C:\Users\Kevin\anaconda3\envs\pyvizenv\lib\site-packages\hvplot\plotting\core.py", line 79, in __call__
 return self._get_converter(x, y, kind, **kwds)(kind, x, y)
 File "C:\Users\Kevin\anaconda3\envs\pyvizenv\lib\site-packages\hvplot\converter.py", line 1137, in __call__
 obj = method(x, y)
 File "C:\Users\Kevin\anaconda3\envs\pyvizenv\lib\site-packages\hvplot\converter.py", line 1524, in bar
 data, x, y = self._process_chart_args(data, x, y, categories=self.by)
 File "C:\Users\Kevin\anaconda3\envs\pyvizenv\lib\site-packages\hvplot\converter.py", line 1419, in _process_chart_args
 elif x in data.columns:
 File "C:\Users\Kevin\anaconda3\envs\pyvizenv\lib\site-packages\pandas\core\indexes\base.py", line 4572, in __contains__
 hash(key)
TypeError: unhashable type: 'list'

Column
    [0] Markdown(str)
    [1] Tabs
        [0] Column(width=600)
            [0] Markdown(str)
            [1] Column
                [0] Column
                    [0] TextInput(name='Your Address', placeholder='Please enter y...)
                    [1] TextInput(name='Your Address', placeholder='Please enter y...)
                [1] Row
                    [0] Str(None, name='interactive02703')
        [1] Column(width=600)
            [0] Markdown(str)
            [1] Column
                [0] Column
                    [0] FileInput(accept='.csv')
                [1] Row
                    [0] Str(None, name='interactive02713')

In [75]:
property_top_level("unit 15 10 12 third avenue campsie", "100000")

[100000, 100000, 100000, 100000, 100000, 100000]
   Years  Median Sold Price  Number of Sales  Purchase Price
0   2016           570000.0              159          100000
1   2017           610000.0              182          100000
2   2018           619000.0              138          100000
3   2019           590000.0               95          100000
4   2020           580000.0              142          100000
5   2021           590000.0              180          100000


TypeError: unhashable type: 'list'